In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import tensorflow as tf
gpu=tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [2]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

lm_list = []
label = "S"
number_of_frames = 600

In [3]:
def make_landmark_timestep(results):
    c_lm = []
    for id, lm in enumerate(results.multi_hand_landmarks[0].landmark):
        if id==0:
            continue
        else:
            c_lm.append(lm.x)
            c_lm.append(lm.y)
            c_lm.append(lm.z)
    return c_lm

In [4]:
cap = cv2.VideoCapture(0)
cap1= cv2.imread('ASL.jpg')
with mp_hands.Hands(
  model_complexity=0,
  min_detection_confidence=0.5,
  min_tracking_confidence=0.5) as hands:

  while len(lm_list) < number_of_frames:
    success, image = cap.read()
    image = cv2.flip(image, 1)
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
                
        lm = make_landmark_timestep(results)
        # lm_list.append(lm)
        if cv2.waitKey(20)&0xFF==ord('z'):
          lm_list.append(lm)
        cv2.putText(image,str(len(lm_list)),(20,30),cv2.FONT_HERSHEY_COMPLEX,1,(250,0,0),1)

    cv2.imshow('Get HandSign', image)
    cv2.imshow("ASL",cap1)

    if cv2.waitKey(5) & 0xFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()

In [5]:
df  = pd.DataFrame(lm_list)
df.to_csv(label + ".txt")

In [6]:
cap.release()
cv2.destroyAllWindows()